<a href="https://colab.research.google.com/github/shabal1n/credit_scoring/blob/main/Credit_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

df = pd.read_excel("Scoring_9.xlsx")
df.head()

,startdate,totalamount,monthly_payment,paymentperiod,collateral_type,flag_sozaem,ip_sign,sum_monthlypayment,sum_outstandingamount,max_dpd_before_startdate,...,incomes,incomes_bdki,work_experience,incomes_bdki_int,incomes_bdki_int_indxd,age,dir,credit_history_length,term,GB_flag
0,2021-04-29,19200000.0,150761.71,578,604,0,0,194760.9478,558508.0,168.0,...,233333.3333,100 000 - 150 000 тенге в месяц,NaN,150000.0,165000.0,50.11,0.83,91,4658,0
1,2021-06-14,5600000.0,76349.66,578,604,0,0,0.0000,0.0,2.0,...,524960.0000,150 000 - 200 000 тенге в месяц,39.0,200000.0,220000.0,34.19,0.00,132,3652,0
2,2020-02-20,11200000.0,110187.75,578,604,0,0,175000.0000,2100000.0,NaN,...,884964.8000,NaN,67.0,NaN,NaN,33.91,0.20,112,5479,0
3,2020-07-13,16000000.0,0.00,578,604,0,0,198537.6190,92204.0,NaN,...,241514.2857,NaN,NaN,NaN,NaN,28.49,0.82,81,5478,0
4,2021-05-19,9500000.0,167892.00,578,604,0,0,0.0000,0.0,2624.0,...,271900.0000,NaN,27.0,NaN,NaN,36.11,0.00,180,3652,0


In [3]:
# Data preprocessing
median_columns = [
    'gender', 'dependants', 
    'maritalstatus', 
    'incomes', 'work_experience',
    'incomes_bdki_int', 
    'incomes_bdki_int_indxd'
    ]
mean_columns = [
    'sum_outstandingamount',
    'max_dpd_before_startdate'
]    
df['startdate'] = pd.to_numeric(pd.DatetimeIndex(df['startdate']).year)
df[df['max_dpd_before_startdate'] == -9999.0] = 0
df = df.drop(['incomes_bdki'], axis=1, errors='ignore')
df = df.drop(['cnt_7d'], axis=1, errors='ignore')
df = df.drop(['cnt_30d'], axis=1, errors='ignore')

for i in median_columns:
  df[i] = df[i].fillna(df[i].median())

for i in mean_columns:
  df[i] = df[i].fillna(df[i].mean())

df.isna().sum()
df.head()

,startdate,totalamount,monthly_payment,paymentperiod,collateral_type,flag_sozaem,ip_sign,sum_monthlypayment,sum_outstandingamount,max_dpd_before_startdate,...,cnt_90d,incomes,work_experience,incomes_bdki_int,incomes_bdki_int_indxd,age,dir,credit_history_length,term,GB_flag
0,2021,19200000.0,150761.71,578,604,0,0,194760.9478,558508.0,168.000000,...,4,233333.3333,46.0,150000.0,165000.0,50.11,0.83,91,4658,0
1,2021,5600000.0,76349.66,578,604,0,0,0.0000,0.0,2.000000,...,3,524960.0000,39.0,200000.0,220000.0,34.19,0.00,132,3652,0
2,2020,11200000.0,110187.75,578,604,0,0,175000.0000,2100000.0,222.245094,...,3,884964.8000,67.0,200000.0,240000.0,33.91,0.20,112,5479,0
3,2020,16000000.0,0.00,578,604,0,0,198537.6190,92204.0,222.245094,...,1,241514.2857,46.0,200000.0,240000.0,28.49,0.82,81,5478,0
4,2021,9500000.0,167892.00,578,604,0,0,0.0000,0.0,2624.000000,...,4,271900.0000,27.0,200000.0,240000.0,36.11,0.00,180,3652,0


In [4]:
def calc_woe_iv(df, target, bins=28, min_samples=0.05):
    result = {}
    
    for col in df.columns:
        if col == target:
            continue
        
        bins_list = np.linspace(df[col].min(), df[col].max(), bins+1)
        
        df['bin'] = pd.cut(df[col], bins_list, labels=False)
        grouped = df.groupby('bin').agg({target: ['count', 'sum']})
        grouped.columns = ['count', 'good']
        grouped['bad'] = grouped['count'] - grouped['good']
        grouped['total'] = grouped['good'] + grouped['bad']
        grouped['prop_good'] = grouped['good'] / grouped['good'].sum()
        grouped['prop_bad'] = grouped['bad'] / grouped['bad'].sum()
        grouped['woe'] = np.log(grouped['prop_good'] / grouped['prop_bad'])
        grouped['iv'] = (grouped['prop_good'] - grouped['prop_bad']) * grouped['woe']
        
        total_iv = grouped['iv'].sum()
        
        if total_iv < min_samples:
            continue
        
        result[col] = {'woe': grouped['woe'], 'iv': total_iv}
        
        df.drop('bin', axis=1, inplace=True)
    
    result_df = pd.DataFrame(result).transpose()
    return result_df.sort_values(by=['iv'], ascending=True)

result_df = calc_woe_iv(df, target='GB_flag')

/usr/local/lib/python3.9/dist-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.9/dist-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.9/dist-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.9/dist-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.9/dist-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.9/dist-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(uf

In [5]:
result_df

,woe,iv
collateral_type,bin 26.0 -0.038064 27.0 1.809780 Name: wo...,0.068514
incomes_bdki_int,bin 0.0 -0.579244 1.0 -0.474932 2.0 -...,0.205545
term,bin 0.0 2.015224 1.0 0.898359 2.0 ...,0.405076
totalamount,bin 0.0 0.042298 1.0 -0.262615 2.0 ...,inf
monthly_payment,bin 0.0 -0.021836 1.0 1.858776 2.0 ...,inf
sum_monthlypayment,bin 0.0 -0.012895 1.0 0.178839 2.0 ...,inf
sum_outstandingamount,bin 0.0 -0.089015 1.0 0.834176 2.0 ...,inf
max_dpd_before_startdate,bin 0.0 -0.059950 1.0 2.645565 27.0 ...,inf
dependants,bin 1.0 -0.091280 2.0 -0.153927 4.0 ...,inf
cnt_90d,bin 0.0 -0.532026 1.0 0.447190 2.0 ...,inf


In [6]:
features = result_df[:15].index.values

In [7]:
y = df.loc[:,'GB_flag']
X = df[features]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
from sklearn.metrics import make_scorer, roc_auc_score
def gini_normalized(y_true, y_pred):
    gini = lambda a, p: 2 * roc_auc_score(a, p) - 1
    return gini(y_true, y_pred) / gini(y_true, y_true)

In [9]:
from scipy.stats import ks_2samp
def ks_statistic(y_true, y_pred):
    ks, _ = ks_2samp(y_pred[y_true==1], y_pred[y_true!=1])
    return ks

In [10]:
print(len(df[df['GB_flag'] == 0]))
print(len(df[df['GB_flag'] == 1]))

24391
609


In [11]:
def log_reg():
  model = LogisticRegression(C=0.01, penalty='l1', solver='liblinear', class_weight='balanced', max_iter=10)

  model.fit(X_train, y_train)

  y_pred = model.predict(X_test)
  print('Logistic regression scores:')
  print('Gini score: %.3f ' % (np.mean(gini_normalized(y_test, y_pred))))
  print('Kolmogorov-Smirnov: %.3f ' % (np.mean(ks_statistic(y_test, y_pred))))

In [12]:
def rand_forest():
  model = RandomForestClassifier(n_estimators=100, random_state=42, verbose=1,
                                      class_weight='balanced', max_features = 0.5, 
                                       min_samples_leaf = 100)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  print('Random Forest Classifier scores:')
  print('Gini score: %.3f ' % (np.mean(gini_normalized(y_test, y_pred))))
  print('Kolmogorov-Smirnov: %.3f ' % (np.mean(ks_statistic(y_test, y_pred))))
  # feature_importance_values = model.feature_importances_
  # feature_importances = pd.DataFrame({'feature': features, 'importance': feature_importance_values})
  # feature_importances.sort_values(by='importance', ascending=False).head(n=10)
  # print(feature_importances)

In [13]:
def lgbm_model():
    params = {
        "objective" : "binary",
        "metric" : "auc",
        "num_leaves" : 1000,
        "learning_rate" : 0.01,
        "bagging_fraction" : 0.8,
        "feature_fraction" : 0.8,
        "bagging_freq" : 5,
        "reg_alpha" : 1.728910519108444,
        "reg_lambda" : 4.9847051755586085,
        "random_state" : 42,
        "bagging_seed" : 2019,
        "verbosity" : -1,
        "max_depth": 18,
        "min_child_samples":100,
        "boosting":"gbdt"
    }
    
    lgtrain = lgb.Dataset(X_train, label=y_train)
    lgval = lgb.Dataset(X_test, label=y_test)
    evals_result = {}
    model = lgb.train(params, lgtrain, 2500, valid_sets=[lgval], 
                      early_stopping_rounds=50, verbose_eval=50, evals_result=evals_result)
    y_pred = model.predict(X_test)
    print('LGBM scores:')
    print('Gini score: %.3f ' % (np.mean(gini_normalized(y_test, y_pred))))
    print('Kolmogorov-Smirnov: %.3f ' % (np.mean(ks_statistic(y_test, y_pred))))

In [14]:
log_reg()

Logistic regression scores:
Gini score: 0.533 
Kolmogorov-Smirnov: 0.533 


/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [15]:
rand_forest()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Random Forest Classifier scores:
Gini score: 0.579 
Kolmogorov-Smirnov: 0.579 


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    5.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished


In [16]:
lgbm_model()

/usr/local/lib/python3.9/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.9/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.9/dist-packages/lightgbm/engine.py:260: UserWarning: 'evals_result' argument is deprecated and will be removed in a future release of LightGBM. Pass 'record_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'evals_result' argument is deprecated and w

Training until validation scores don't improve for 50 rounds
[50]	valid_0's auc: 0.869752
[100]	valid_0's auc: 0.874666
[150]	valid_0's auc: 0.876916
[200]	valid_0's auc: 0.877031
Early stopping, best iteration is:
[167]	valid_0's auc: 0.877541
LGBM scores:
Gini score: 0.755 
Kolmogorov-Smirnov: 0.638 
